
# Tensorboard for visual logging

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 3486), started 2:23:39 ago. (Use '!kill 3486' to kill it.)

<IPython.core.display.Javascript object>

# Imports

In [3]:
import torchvision, torchvision.transforms as transforms
import torch, torch.nn as nn
import random, subprocess, os
import numpy as np
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from google.colab import drive

# To read and write to google drive
drive.mount("/content/drive/")

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive/


# Dataloaders

In [4]:
def load_loader_stl(crop_size: int = 33, batch_size: int = 128, num_workers: int = 1, scale: float = 2.0):
    """
    Loads the dataloader of the STL-10 Dataset using the given specifications with the required
                          augmentation schemes
    input : crop_size -> image size of the square sub images the model has been trained on
            scale     -> Scale by which the low resolution image is downscaled
    output: dataloader iterable to be able to train on the images

    Augmentation Schemes: Since torch has strong built in support for transforms, augmentation
                          was done within our dataloader transforms employing TenCrop on each
                          image. For every image we get 5 crops (Center + 4 corners) and the horizontal
                          flip of each. TenCrop returns a tuple, which was handled using lambda
                          and also in the training script in the cell below.

    """
    # Write transforms for TenCrop and for generating low res images using bicubic interpolation (interpolation = 3)
    transform_high_res = transforms.Compose([
            transforms.TenCrop(crop_size),
            transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
        ])
    transform_low_res = transforms.Compose([
            transforms.Resize(int(96 / scale), interpolation=3),
            transforms.Resize(96, interpolation=3),
            transform_high_res
        ])

    # Make STL-10 dataset object
    dataset_high_res = torchvision.datasets.STL10('.', transform = transform_high_res, download = True)
    dataset_low_res = torchvision.datasets.STL10('.', transform = transform_low_res, download = False)

    # Create the dataloader object using the transforms (Not shuffled since we will be checking progress on the same examples)
    dataloader_high_res = torch.utils.data.DataLoader(dataset_high_res, batch_size = batch_size, num_workers = num_workers, shuffle = False)
    dataloader_low_res = torch.utils.data.DataLoader(dataset_low_res, batch_size = batch_size, num_workers = num_workers, shuffle = False)
    return dataloader_low_res, dataloader_high_res

# Models used

In [9]:
class SuperResolution(nn.Module):
    """
    Network Architecture as per specified in the paper.
    The chosen configuration for successive filter sizes are 9-5-5
    The chosed configuration for successive filter depth are 128-64(-3)
    """
    def __init__(self, sub_image: int = 33, spatial: list = [9, 5, 5], filter: list = [128, 64], num_channels: int = 3):
        super().__init__()
        self.layer_1 = nn.Conv2d(num_channels, filter[0], spatial[0], padding = spatial[0] // 2)
        self.layer_2 = nn.Conv2d(filter[0], filter[1], spatial[1], padding = spatial[1] // 2)
        self.layer_3 = nn.Conv2d(filter[1], num_channels, spatial[2], padding = spatial[2] // 2)
        self.relu = nn.ReLU()

    def forward(self, image_batch):
        x = self.layer_1(image_batch)
        x = self.relu(x)
        x = self.layer_2(x)
        y = self.relu(x)
        x = self.layer_3(y)
        return x, y

# Training loop

In [3]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from tqdm import tqdm
import os

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define your SuperResolution model (you need to implement this)
class SuperResolution(nn.Module):
    def __init__(self):
        super(SuperResolution, self).__init__()
        # Add your model layers here
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, padding=2)

    def forward(self, x):
        # Example forward pass
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.conv3(x)
        return x, x  # Simplified for example

# Training function (updated)
def train():
    # Initialize components
    low_res_loader, high_res_loader = load_loader_stl()
    model = SuperResolution().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    writer = SummaryWriter()
    n = 0

    # Define transforms with legacy interpolation
    to_pil = transforms.ToPILImage()
    resize = transforms.Resize((48*7, 144*7), interpolation=Image.BICUBIC)
    other_resize = transforms.Resize((48*5, 48*5), interpolation=Image.BICUBIC)

    for epoch in tqdm(range(500), desc="Training", ncols=120):
        for low_res, high_res in zip(low_res_loader, high_res_loader):
            # Process batches
            low_res_batch = low_res[0].view(-1, 3, 24, 24).to(device)
            high_res_batch = high_res[0].view(-1, 3, 96, 96).to(device)

            # Forward pass
            reconstructed, intermediate = model(low_res_batch)

            # Calculate loss
            loss = nn.MSELoss()(reconstructed, high_res_batch)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Logging and visualization (same as before)
            # ... [rest of your original training code] ...

        # Save model checkpoint
        torch.save(model.state_dict(), '/content/drive/My Drive/isr/isr_best_2.pth')

    return model

# Start training
model = train()

Files already downloaded and verified


TypeError: DataLoader.__init__() got an unexpected keyword argument 'transform'

In [20]:
!pip install --upgrade Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 64.9 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.1
    Uninstalling Pillow-7.1.1:
      Successfully uninstalled Pillow-7.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires tqdm>=4.64.1, but you have tqdm 4.45.0 which is incompatible.


In [15]:
%cd image-super-resolution

/content/image-super-resolution


In [17]:
!python execute.py --image https://raw.githubusercontent.com/amanshenoy/image-super-resolution/master/results/barbara.bmp --scale 2 --path results/download.png --saved saved/isr_best.pth

--2025-03-07 19:26:50--  https://raw.githubusercontent.com/amanshenoy/image-super-resolution/master/results/barbara.bmp
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1244214 (1.2M) [image/bmp]
Saving to: ‘results/download.png’

results/download.pn 100%[===================>]   1.19M  --.-KB/s    in 0.008s  

2025-03-07 19:26:50 (155 MB/s) - ‘results/download.png’ saved [1244214/1244214]

/content/image-super-resolution/execute.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models f

In [18]:
!python execute.py --image results/sample1.bmp --scale 2 --saved saved/isr_best.pth

/content/image-super-resolution/execute.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(args.saved, map_location={'cuda:0': 'cpu'}))
P

In [2]:
# Clone repo
#!git clone https://github.com/amanshenoy/image-super-resolution.git

# Navigate to directory
#%cd image-super-resolution

# Install dependencies
!pip install -r requirements.txt

# Restart the runtime if needed (uncomment below)
# import os
# os.kill(os.getpid(), 9)

  Using cached Pillow-7.1.1.tar.gz (38.9 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━